# 1. Import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_auc_score, confusion_matrix
from IPython.display import clear_output

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import tensorflow as tf

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

# 2. Import datasets

In [4]:
df = pd.read_csv("../01 raw data/05_sampling.csv")
df = df[['transactionID','Fraud','b_accountAge','b_transactionAmountUSD','b_ipCountryCode','b_ipState',
         'b_paymentInstrumentAgeInAccount','b_accountCountry','b_cardType','b_transactionCurrencyCode','b_localHour']]
print(df.shape)

(101207, 11)


In [5]:
df0 = df[df['Fraud']==0].reset_index(drop=True)
df1 = df[df['Fraud']==1].reset_index(drop=True)

df01 = df0[:][:31220]
df02 = df0[:][31220:62440]
df03 = df0[:][62440:]

df11 = df1[:][:2510]
df12 = df1[:][2510:5020]
df13 = df1[:][5020:]

df1 = pd.concat([df01,df11])
df2 = pd.concat([df02,df12])
df3 = pd.concat([df03,df13])

print(df1.shape, df2.shape, df3.shape)

(33730, 11) (33730, 11) (33747, 11)


# 3. Model - 1st

In [6]:
np.random.seed(108)
activation_c = []
optimizer_c = []
l2_c = []
epochs_c = []
batch_size_c = []
auc_c = []

In [7]:
for a in ['relu']:
    for o in ['adam']:
        for l in [0.001, 0.1]:
            for e in [100, 1000]:
                for b in [100, 1000]:
                    model = Sequential() 
                    model.add(Dense(6, activation=a, input_dim=9, kernel_regularizer=l2(l))) 
                    model.add(Dense(6, activation=a, kernel_regularizer=l2(l)))
                    model.add(Dense(6, activation=a, kernel_regularizer=l2(l)))
                    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l)))
                    model.compile(optimizer=o, loss='binary_crossentropy', metrics='AUC') 

                    h = []
                    for i in [0,1,2]:
                        if i==0: train=pd.concat([df1,df2])
                        elif i==2: train=pd.concat([df2,df3])
                        else: train=pd.concat([df3,df1])
                        y = train['Fraud']
                        x = train.drop(['transactionID','Fraud'], axis=1)
                        model.fit(x,y,epochs=e,batch_size=b,verbose=0,callbacks=EarlyStopping(monitor='loss',patience=3))

                        if i==0: test=df3
                        elif i==2: test=df1
                        else: test=df2
                        y = test['Fraud']
                        x = test.drop(['transactionID','Fraud'], axis=1)
                        pred_values = model.predict(x,batch_size=b,verbose=0)

                        pred_values = pd.DataFrame(pred_values)[0]
                        h.append(np.round(roc_auc_score(y, pred_values), 3))
                    clear_output(wait=False)
                    print('activation\toptimizer\tl2\tepochs\tbatch_size\tauc')
                    print(a,'\t\t',o,'\t\t',l,'\t',e,'\t',b,'\t\t',np.round(np.mean(h), 3))

                    activation_c.append(a)
                    optimizer_c.append(o)
                    l2_c.append(l)
                    epochs_c.append(e)
                    batch_size_c.append(b)
                    auc_c.append(np.round(np.mean(h), 3))

activation	optimizer	l2	epochs	batch_size	auc
relu 		 adam 		 0.1 	 1000 	 1000 		 0.5


In [8]:
train = pd.DataFrame({'activation':activation_c, 'optimizer':optimizer_c, 'l2':l2_c,
                      'epochs':epochs_c, 'batch_size':batch_size_c, 'auc':auc_c})
test = train[train['auc']==train['auc'].max()]
test

,activation,optimizer,l2,epochs,batch_size,auc
3,relu,adam,0.001,1000,1000,0.738


# 4. Model - 2nd

In [9]:
np.random.seed(108)
activation_c = []
optimizer_c = []
l2_c = []
epochs_c = []
batch_size_c = []
auc_c = []

In [10]:
for a in ['relu']:
    for o in ['adam']:
        for l in [0.001, 0.1]:
            for e in [100, 1000]:
                for b in [100, 1000]:
                    model = Sequential() 
                    model.add(Dense(3, activation=a, input_dim=9, kernel_regularizer=l2(l))) 
                    model.add(Dense(3, activation=a, kernel_regularizer=l2(l)))
                    model.add(Dense(3, activation=a, kernel_regularizer=l2(l)))
                    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l)))
                    model.compile(optimizer=o, loss='binary_crossentropy', metrics='AUC') 

                    h = []
                    for i in [0,1,2]:
                        if i==0: train=pd.concat([df1,df2])
                        elif i==2: train=pd.concat([df2,df3])
                        else: train=pd.concat([df3,df1])
                        y = train['Fraud']
                        x = train.drop(['transactionID','Fraud'], axis=1)
                        model.fit(x,y,epochs=e,batch_size=b,verbose=0,callbacks=EarlyStopping(monitor='loss',patience=3))

                        if i==0: test=df3
                        elif i==2: test=df1
                        else: test=df2
                        y = test['Fraud']
                        x = test.drop(['transactionID','Fraud'], axis=1)
                        pred_values = model.predict(x,batch_size=b,verbose=0)

                        pred_values = pd.DataFrame(pred_values)[0]
                        h.append(np.round(roc_auc_score(y, pred_values), 3))
                    clear_output(wait=False)
                    print('activation\toptimizer\tl2\tepochs\tbatch_size\tauc')
                    print(a,'\t\t',o,'\t\t',l,'\t',e,'\t',b,'\t\t',np.round(np.mean(h), 3))

                    activation_c.append(a)
                    optimizer_c.append(o)
                    l2_c.append(l)
                    epochs_c.append(e)
                    batch_size_c.append(b)
                    auc_c.append(np.round(np.mean(h), 3))

activation	optimizer	l2	epochs	batch_size	auc
relu 		 adam 		 0.1 	 1000 	 1000 		 0.5


In [11]:
train = pd.DataFrame({'activation':activation_c, 'optimizer':optimizer_c, 'l2':l2_c,
                      'epochs':epochs_c, 'batch_size':batch_size_c, 'auc':auc_c})
test = train[train['auc']==train['auc'].max()]
test

,activation,optimizer,l2,epochs,batch_size,auc
1,relu,adam,0.001,100,1000,0.736


# 5. Model - 3rd

In [12]:
np.random.seed(108)
activation_c = []
optimizer_c = []
l2_c = []
epochs_c = []
batch_size_c = []
auc_c = []

In [13]:
for a in ['relu']:
    for o in ['adam']:
        for l in [0.001, 0.1]:
            for e in [100, 1000]:
                for b in [100, 1000]:
                    model = Sequential() 
                    model.add(Dense(6, activation=a, input_dim=9, kernel_regularizer=l2(l))) 
                    model.add(Dense(6, activation=a, kernel_regularizer=l2(l)))
                    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l)))
                    model.compile(optimizer=o, loss='binary_crossentropy', metrics='AUC') 

                    h = []
                    for i in [0,1,2]:
                        if i==0: train=pd.concat([df1,df2])
                        elif i==2: train=pd.concat([df2,df3])
                        else: train=pd.concat([df3,df1])
                        y = train['Fraud']
                        x = train.drop(['transactionID','Fraud'], axis=1)
                        model.fit(x,y,epochs=e,batch_size=b,verbose=0,callbacks=EarlyStopping(monitor='loss',patience=3))

                        if i==0: test=df3
                        elif i==2: test=df1
                        else: test=df2
                        y = test['Fraud']
                        x = test.drop(['transactionID','Fraud'], axis=1)
                        pred_values = model.predict(x,batch_size=b,verbose=0)

                        pred_values = pd.DataFrame(pred_values)[0]
                        h.append(np.round(roc_auc_score(y, pred_values), 3))
                    clear_output(wait=False)
                    print('activation\toptimizer\tl2\tepochs\tbatch_size\tauc')
                    print(a,'\t\t',o,'\t\t',l,'\t',e,'\t',b,'\t\t',np.round(np.mean(h), 3))

                    activation_c.append(a)
                    optimizer_c.append(o)
                    l2_c.append(l)
                    epochs_c.append(e)
                    batch_size_c.append(b)
                    auc_c.append(np.round(np.mean(h), 3))

activation	optimizer	l2	epochs	batch_size	auc
relu 		 adam 		 0.1 	 1000 	 1000 		 0.5


In [14]:
train = pd.DataFrame({'activation':activation_c, 'optimizer':optimizer_c, 'l2':l2_c,
                      'epochs':epochs_c, 'batch_size':batch_size_c, 'auc':auc_c})
test = train[train['auc']==train['auc'].max()]
test

,activation,optimizer,l2,epochs,batch_size,auc
1,relu,adam,0.001,100,1000,0.733


# 6. Model - 4th

In [15]:
np.random.seed(108)
activation_c = []
optimizer_c = []
l2_c = []
epochs_c = []
batch_size_c = []
auc_c = []

In [16]:
for a in ['relu']:
    for o in ['adam']:
        for l in [0.001, 0.1]:
            for e in [100, 1000]:
                for b in [100, 1000]:
                    model = Sequential() 
                    model.add(Dense(3, activation=a, input_dim=9, kernel_regularizer=l2(l))) 
                    model.add(Dense(3, activation=a, kernel_regularizer=l2(l)))
                    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l)))
                    model.compile(optimizer=o, loss='binary_crossentropy', metrics='AUC') 

                    h = []
                    for i in [0,1,2]:
                        if i==0: train=pd.concat([df1,df2])
                        elif i==2: train=pd.concat([df2,df3])
                        else: train=pd.concat([df3,df1])
                        y = train['Fraud']
                        x = train.drop(['transactionID','Fraud'], axis=1)
                        model.fit(x,y,epochs=e,batch_size=b,verbose=0,callbacks=EarlyStopping(monitor='loss',patience=3))

                        if i==0: test=df3
                        elif i==2: test=df1
                        else: test=df2
                        y = test['Fraud']
                        x = test.drop(['transactionID','Fraud'], axis=1)
                        pred_values = model.predict(x,batch_size=b,verbose=0)

                        pred_values = pd.DataFrame(pred_values)[0]
                        h.append(np.round(roc_auc_score(y, pred_values), 3))
                    clear_output(wait=False)
                    print('activation\toptimizer\tl2\tepochs\tbatch_size\tauc')
                    print(a,'\t\t',o,'\t\t',l,'\t',e,'\t',b,'\t\t',np.round(np.mean(h), 3))

                    activation_c.append(a)
                    optimizer_c.append(o)
                    l2_c.append(l)
                    epochs_c.append(e)
                    batch_size_c.append(b)
                    auc_c.append(np.round(np.mean(h), 3))

activation	optimizer	l2	epochs	batch_size	auc
relu 		 adam 		 0.1 	 1000 	 1000 		 0.5


In [17]:
train = pd.DataFrame({'activation':activation_c, 'optimizer':optimizer_c, 'l2':l2_c,
                      'epochs':epochs_c, 'batch_size':batch_size_c, 'auc':auc_c})
test = train[train['auc']==train['auc'].max()]
test

,activation,optimizer,l2,epochs,batch_size,auc
0,relu,adam,0.001,100,100,0.731
3,relu,adam,0.001,1000,1000,0.731
